In [15]:
%load_ext autoreload
%autoreload 2
from module_test.raw_code.optionlib_2.assets.forward import (
    Forward,
    EquityForward,
    vectorized_forward_discrete,
    vectorized_forward_continuous,
    time_distance_helper
)

from module_test.raw_code.optionlib_2.assets.dividend import (
    MarketContinuousDividends,
    MarketDividendSchedule,
    vectorized_discrete_pv,
    get_vectorized_dividend_scehdule,
    get_vectorized_dividend_rate,
    get_vectorized_continuous_dividends
)
from datetime import datetime

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
from module_test.raw_code.optionlib_2.pricing.binomial import Node

In [20]:
tree = []
for i in range(3):
    tree.append([])
    for j in range(i + 1):
        node = Node(
            0, 1, (i,j)
        )
        tree[i].append(node)

In [32]:
class Empty:
    def __init__(self, *args):
        self.args = args
    def __repr__(self):
        return f"Empty{self.args}"

In [33]:
import numpy as np
test_scheduel = \
[
    [Empty(1,1), Empty(2,1), Empty(3,1)],
    [Empty(1,2), Empty(2,2), Empty(3,2)],
    [Empty(1,3), Empty(2,3), Empty(3,3)],
]

np.array(test_scheduel, dtype=object)

array([[Empty(1, 1), Empty(2, 1), Empty(3, 1)],
       [Empty(1, 2), Empty(2, 2), Empty(3, 2)],
       [Empty(1, 3), Empty(2, 3), Empty(3, 3)]], dtype=object)

In [2]:
forward = Forward(
    start_date="2023-01-01",
    end_date="2024-01-01",
    spot_price=100,
    risk_free_rate=0.05,
    dividend_type="discrete",
    freq="quarterly",
    div_amount=[0, 0, 0, 0]
)
print(forward.get_forward_price())  # Example usage
print(forward.dividend.get_present_value(forward.risk_free_rate, sum_up=True))
print(forward.dividend.get_present_value(forward.risk_free_rate, sum_up=False))
forward

105.12351191991695
0.0
[0.0, 0.0, 0.0, 0.0]


<Forward: ({'spot': 100, 'forward': 105.12351191991695, 'type': 'discrete', 'valuation': datetime.date(2023, 1, 1), 'expiry': datetime.date(2024, 1, 1)}>

In [3]:
forward_continous = Forward(
    start_date="2023-01-01",
    end_date="2024-01-01",
    spot_price=100,
    risk_free_rate=0.05,
    dividend_type="continuous",
    div_amount=0.0  # 2% continuous dividend yield
)
print(forward_continous.get_forward_price())  # Example usage

105.12351191991695


#### Model (Tickers)

- This section carries out market data injections. Instead of relying on user inputs for

In [9]:
forward = EquityForward(
    start_date="2023-01-03",
    end_date="2024-01-03",
    risk_free_rate=None,
    # risk_free_rate=0.05,
    dividend_type="discrete",
    ticker = 'BAC'
)
print(forward.get_forward_price())  # Example usage
print(forward.dividend.get_present_value(forward.risk_free_rate, sum_up=True))
print(forward.dividend.get_present_value(forward.risk_free_rate, sum_up=False))
forward

34.03482883323706
0.893653476627657
[0.21854228010864224, 0.22175365892120283, 0.22501222747605323, 0.2283453101217587]


<Forward: ({'spot': 33.5099983215332, 'forward': 34.03482883323706, 'type': 'discrete', 'valuation': datetime.date(2023, 1, 3), 'expiry': datetime.date(2024, 1, 3)}>

In [11]:
forward.dividend.asset.list_instances()

{('AAPL',
  '2023-01-03 16:00:00'): Stock(Ticker: AAPL, Build Date: 2023-01-03 16:00:00),
 ('BAC',
  '2023-01-03 16:00:00'): Stock(Ticker: BAC, Build Date: 2023-01-03 16:00:00)}

In [5]:
forward.dividend.asset.rf_rate
forward.risk_free_rate
forward.dividend.amounts

[0.23, 0.2329913294797688, 0.23602156353369644, 0.23909120814612894]

In [6]:
forward.dividend.spot_price

125.06999969482422

In [7]:
forward_continous = EquityForward(
    start_date="2023-01-03",
    end_date="2024-01-03",
    risk_free_rate=0.05,
    dividend_type="continuous",
    div_amount=0.0,  # 2% continuous dividend yield
    ticker = 'AAPL'
)
print(forward_continous.get_forward_price())  # Example usage

130.75511472843496


In [8]:
forward_continous.dividend.spot_price 
forward_continous.dividend.yield_rate  

0.005516910543564624

In [13]:
tickers = ['TSLA', 'CVX', 'KO']
r = [0.045, 0.055, 0.005]
spots=[100,200,300]
start_dates = [datetime(2025, 1, 5), datetime(2025, 1, 5), datetime(2025, 1, 5)]
end_dates = [datetime(2025, 8, 31), datetime(2025, 8, 31), datetime(2025, 8, 31)]
valuation_dates = [datetime(2025, 1, 5), datetime(2025, 1, 5), datetime(2025, 1, 5)]
schedules = get_vectorized_dividend_scehdule(
    tickers=tickers,
    start_dates=start_dates,
    end_dates=end_dates,
    valuation_dates=valuation_dates
)
discrete_pv = vectorized_discrete_pv(schedules, 
                                     r,
                                     _valuation_dates=valuation_dates,
                                     _end_dates=end_dates)
discrete_pv


div_rates = get_vectorized_dividend_rate(
    tickers=tickers,
    spots=spots,
    valuation_dates=valuation_dates
)
cont_q = get_vectorized_continuous_dividends(
    div_rates=div_rates,
    _valuation_dates=valuation_dates,
    _end_dates=end_dates

)
cont_q

2025-07-23 23:00:50 trade.optionlib.utils.market_data ERROR: Error fetching dividend schedule for TSLA: 
[Error] -> Error getting data for TSLA: No dividend data found for TSLA


array([1.        , 0.97898159, 0.99579513])

In [16]:
tickers = ['TSLA', 'CVX', 'KO']
r = [0.045, 0.055, 0.005]
spots=[100,200,300]
start_dates = [datetime(2025, 1, 5), datetime(2025, 1, 5), datetime(2025, 1, 5)]
end_dates = [datetime(2025, 8, 31), datetime(2025, 8, 31), datetime(2025, 8, 31)]
valuation_dates = [datetime(2025, 1, 5), datetime(2025, 1, 5), datetime(2025, 1, 5)]
vector_f_discrete = vectorized_forward_discrete(
    S=spots,
    r=r,
    T=[time_distance_helper(end_dates[i], valuation_dates[i]) for i in range(len(end_dates))],
    pv_divs=discrete_pv
)

vector_f_continuous = vectorized_forward_continuous(
    S=spots,
    r=r,
    q_factor=cont_q,
    T=[time_distance_helper(end_dates[i], valuation_dates[i]) for i in range(len(end_dates))]
)

vector_f_continuous, vector_f_discrete

(array([102.97565159, 202.94061447, 299.71342922]),
 array([102.97565159, 203.86358679, 299.98726676]))